In [7]:
from pathlib import Path
import json
import openai
from datetime import datetime
from pydantic import BaseModel

In [9]:
key_path = Path("openai_key.txt")
if not key_path.exists():
    raise RuntimeError("Missing openai_key.txt; please create it with your API key")
openai.api_key = key_path.read_text().strip()

In [11]:
product_input = 'Introducing the all-new UltraLite Camping Stove — your perfect companion for outdoor adventures. Crafted from anodized aluminum and stainless steel, it weighs just 850g and measures 12x12x10 cm when folded. Now with free shipping on orders over €50! Designed for high-altitude cooking, it includes a piezo ignition system, adjustable flame control, and supports pots up to 20cm wide. SKU: ULCS-2024. Price: €79.95. Note: Due to demand, delivery may take 5–7 business days. Join over 10,000 happy campers who trust our gear! Also, check out our new Instagram page for adventure tips and exclusive promos!'

In [13]:
example_product = {
  "product_id": "string",
  "product_name":"string",
  "category": "string",              
  "sub_category": "string",          
  "region": "string",                
  "marketplace": "string",           
  "attributes": {
    "size": {                        
      "value": 365,
      "unit": "cm"
    },
    "color": "Matte Black",
    "material": "Ceramic",
    "finish": "Glossy",
    "capacity_ml": 330               
  },
  "pricing": {
    "cost": {
      "amount": 2.50,
      "currency_code": "EUR"
    },
    "msrp": {
      "amount": 5.00,
      "currency_code": "EUR"
    }
  },
  "inventory": {
    "stock_level": 120,
    "reorder_threshold": 20
  },
  "supplier_info": {
    "vendor_id": "VendorName",
    "vendor_sku": "SKU-X123"
  },
  "addition_information":{}, 
  "metadata": {
    "source_vendor": "VendorName",
    "language": "en-US",
    "last_updated": "datetime",
    "tags": ["mug","ceramic","drinkware"]
  }
}

# schema = json.dumps(example_product, indent=2)

# print(schema)

In [15]:
functions = [
    {
        "name": "get_date",
        "description": "Retrives the current date",
        "parameters": {
            "type": "object",
            "properties": {
                "current_date": {
                    "type": "string",
                    "description": "Today's date in ISO 8601 format"
                }
            },
            "required": ["current_date"]
        }
    }
]

today = datetime.today().strftime("%Y-%m-%dT%H:%M:%SZ")


function_response = {
    "name": "get_date",
    "arguments": {
        "current_date": today
    }
}

In [19]:
client = openai.OpenAI(
  api_key=openai.api_key)

extraction = client.responses.create(
  model="gpt-4o",
  instructions="""
  You are an extractor assistant, you will peform the following tasks:
  1) review the inputs given to you and generate json data where only product information is extracted wherever you find relevant. 
  2) produce a confidence score to assess how confident you find the results.
  """,
  input=product_input,
  store=True
)

raw_input = product_input
extracted_data = extraction.output_text
print(raw_input)
print(extracted_data)


Introducing the all-new UltraLite Camping Stove — your perfect companion for outdoor adventures. Crafted from anodized aluminum and stainless steel, it weighs just 850g and measures 12x12x10 cm when folded. Now with free shipping on orders over €50! Designed for high-altitude cooking, it includes a piezo ignition system, adjustable flame control, and supports pots up to 20cm wide. SKU: ULCS-2024. Price: €79.95. Note: Due to demand, delivery may take 5–7 business days. Join over 10,000 happy campers who trust our gear! Also, check out our new Instagram page for adventure tips and exclusive promos!
```json
{
  "product": {
    "name": "UltraLite Camping Stove",
    "materials": ["anodized aluminum", "stainless steel"],
    "weight_g": 850,
    "dimensions_cm": {
      "folded": [12, 12, 10]
    },
    "features": [
      "Designed for high-altitude cooking",
      "Piezo ignition system",
      "Adjustable flame control",
      "Supports pots up to 20cm wide"
    ],
    "sku": "ULCS-2024

In [21]:
input_with_context = {
    "schema": example_product,
    "context": {
        "current_date": today
    }
}

inputs = json.dumps(input_with_context, indent=2)

# print(inputs)

In [23]:
# mapping = client.responses.create(
#     model="gpt-4o",
#     previous_response_id=extraction.id,
#     instructions="You are a mapping assistant, you will review the extracted data and map it to the schema provided to you, map each entry strictly wherever the data is available, if it is not, first infer from all available data points, if you are unsure then just leave it blank, also generate a randomized 10 character product ID using number & letters and fill last_updated with current_date",
#     input=inputs
# )

# mapped_data = mapping.output_text

# print(mapped_data)

In [25]:
class Step(BaseModel):
    explanation: str
    output: str

class Reasoning(BaseModel):
    steps: list[Step]
    final_answer: str


mapping = client.responses.parse(
    model="gpt-4o",
    previous_response_id=extraction.id,
    instructions="""You are a mapping assistant, you will perform the following tasks:
    1) review the extracted data and map it to the schema provided to you -- map each entry strictly wherever the data is available, if it is not, first infer from all available data points, if you are unsure then just leave it blank, 
    2) generate a randomized 10 character product ID using number & letters 
    3) fill last_updated with current_date.
    """,
    input=inputs,
    text_format=Reasoning,
)

mapping_reasoning = mapping.output_parsed

# print(mapping_reasoning)

In [26]:
trace = {
    "steps": [{"explanation": step.explanation, "output": step.output} for step in mapping_reasoning.steps],
    "final_answer": json.loads(mapping_reasoning.final_answer)
}

print(json.dumps(trace, indent=2))

{
  "steps": [
    {
      "explanation": "Map the product name from input to schema. It's the name of the product provided.",
      "output": "UltraLite Camping Stove"
    },
    {
      "explanation": "Category and sub-category are not provided in the text. The product is a stove, belongs to outdoor gear but since it's not explicit, leave it blank.",
      "output": ""
    },
    {
      "explanation": "Region and marketplace are not specified, so leave these blank.",
      "output": ""
    },
    {
      "explanation": "Attributes like size, color, material, etc., do not match the input details and should remain empty.",
      "output": ""
    },
    {
      "explanation": "Pricing information like cost and MSRP are not included, hence leave these blank.",
      "output": ""
    },
    {
      "explanation": "Inventory details like stock levels and reorder thresholds are not provided, so leave empty.",
      "output": ""
    },
    {
      "explanation": "Supplier information is mis

In [27]:
mapping_eval = client.responses.create(
    model="gpt-4o",
    previous_response_id=mapping.id,
    input="Produce a confidence score to assess how confident you find the mapped results."
)

mapping_score = mapping_eval.output_text
print(mapping_score)

The confidence score for the mapped results is relatively low, around **0.4**, due to the mismatch between the provided schema and the available data. The schema was designed for a different type of product, leading to many fields not being applicable or left empty.


In [28]:
search = client.responses.create(
    model="gpt-4o",
    previous_response_id=extraction.id,
    tools=[{ "type": "web_search_preview" }],
    input="Using the extracted information, perform an extensive online search to find relevant information from manufacturer sites and competitor sites on the given product, show source of the websites too",
)

searched_data = search.output_text

print(searched_data)

The UltraLite Camping Stove (SKU: ULCS-2024) is a compact and lightweight stove designed for outdoor enthusiasts. Crafted from anodized aluminum and stainless steel, it weighs 850 grams and measures 12x12x10 cm when folded. Features include a piezo ignition system, adjustable flame control, and the capacity to support pots up to 20 cm in diameter. Priced at €79.95, the stove is currently available with free shipping on orders over €50, with an estimated delivery time of 5–7 business days.

**Manufacturer Information:**

As of now, specific details about the manufacturer of the UltraLite Camping Stove are not readily available. For comprehensive information, including user manuals, warranty details, and customer support, it is advisable to consult the official product website or contact the retailer directly.

**Competitor Products:**

Several other camping stoves offer similar features and may serve as alternatives to the UltraLite Camping Stove:

1. **MSR PocketRocket Deluxe:**
   - *

In [37]:
class FinalEnrichment(BaseModel):
    enriched_output: str
    confidence_score: float
    change_score: str

final = client.responses.parse(
    model="gpt-4o",
    previous_response_id=mapping.id,
    instructions="""
You are an enrichment assistant. You will receive 4 inputs:
  1) 'schema' - output schema (with example inputs) to be followed
  2) `mapped` – already mapped data
  3) `searched` – information searched online concerning the same product with additional and supplementart information,
  4) 'mapping score' - confidence score of the previously mapped data

Your job is to:
- Access mapped data and searched data, evaluate which entries are missing and are NOT currently mapped
- For missing entries, fill them if you can find corresponding information directly from searched data, if not, you can infer about what the right value might be
- Do **not** modify any fields that are already populated unless the searched data is clearly more up‑to‑date; in that case, overwrite.
- After enriching all records, calculate a single `confidence_score` between 0.0 and 1.0 representing how complete and reliable your enrichment is (1.0 = 100% confident).
- Show mapped data vertically in JSON format.
- Show change (+/-) from original mapping score.
    """,
    input=json.dumps({
        "schema":example_product,
        "mapped": trace["final_answer"],
        "searched": searched_data,
        "mapping_score": mapping_score
    }),
    text_format=FinalEnrichment,
)

results = final.output_parsed.enriched_output
score = final.output_parsed.confidence_score
change_score = final.output_parsed.change_score

print("✅ Mapped and Enriched Records:\n", results)
print("🔢 Confidence Score:", score)
print("📈 Change in original mapping confidence score after enrichment:", change_score)


✅ Mapped and Enriched Records:
 {
  "product_id": "83A7LC95QF",
  "product_name": "UltraLite Camping Stove",
  "category": "Outdoor Gear",
  "sub_category": "Camping Equipment",
  "region": "Europe",
  "marketplace": "Online Retailer",
  "attributes": {
    "size": {
      "value": null,
      "unit": null
    },
    "color": null,
    "material": "Anodized Aluminum, Stainless Steel",
    "finish": null,
    "capacity_ml": null
  },
  "pricing": {
    "cost": {
      "amount": 79.95,
      "currency_code": "EUR"
    },
    "msrp": {
      "amount": null,
      "currency_code": null
    }
  },
  "inventory": {
    "stock_level": null,
    "reorder_threshold": null
  },
  "supplier_info": {
    "vendor_id": null,
    "vendor_sku": "ULCS-2024"
  },
  "addition_information": {
    "features": "Piezo ignition system, Adjustable flame control, Supports pots up to 20cm wide",
    "shipping_info": "Free shipping on orders over €50, Delivery: 5–7 business days"
  },
  "metadata": {
    "source_